In [26]:
import numpy as np
import pandas as pd
import itertools
import copy

from scipy.optimize import fsolve
from scipy.special import gamma

In [27]:
class City(object):
    
    def __init__(self, width, height, hospital_count, subregion_count):
        
        self.width = width
        self.height = height
        #for simplicity, subregion_count = base_count
        
        hospital_x = np.linspace(0, self.width, num = hospital_count + 2)
        self.hospital_location = [(hospital_x[i+1], self.height/2) for i in range(hospital_count)]
        
        self.subregion_count = subregion_count + (subregion_count % 2)
        subregion_x = np.linspace(0, self.width, num = int(self.subregion_count/2 + 2))
        subregion_y = np.linspace(0, self.height, num = 4)
        base_location = [[(subregion_x[i+1], subregion_y[j+1]) for j in range(2)] for i in range(int(self.subregion_count/2))]
        self.base_location = list(itertools.chain(*base_location))

In [28]:
def gen_call_arrival_loc(city):
    return (np.random.uniform(0, city.width),np.random.uniform(0, city.height))

In [29]:
def gen_call_arrival_time():
    return np.random.exponential(3)

In [30]:
def gen_atlocation_service_time():
    return np.random.exponential(12)

In [31]:
def weibull_param_relationship(scale, mean, stddev):
    return stddev**2/mean**2 - gamma(1 + 2/scale)/(gamma(1+1/scale)**2) + 1

def get_weibull_parameters(mean = 30, stddev = 13):
    scale = fsolve(weibull_param_relationship, 1,args=(mean, stddev))
    shape = mean / gamma(1+1/scale)
    
    return scale, shape

def gen_athospital_service_time():
    scale, shape = get_weibull_parameters(mean = 30, stddev = 13)
    return (scale * np.random.weibull(shape))[0]

In [32]:
class Call(object):
    
    def __init__(self, call_index, location, time_arrival, interarrival_time, priority = 0):
        
        self.call_index = call_index
        self.location = location
        
        #initialize the call to be unassigned
        #-1-unassigned, 0 to (N-1) assigned ambulance index 
        self.status = -1
        self.arrival_time = time_arrival
        self.next_arrival_time = self.arrival_time + interarrival_time
        self.priority = priority

In [33]:
def get_distance(location1, location2):
    
    distance = np.abs(location1[0] - location2[0]) + np.abs(location1[1] - location2[1])
    return distance

In [34]:
def get_ambulance_travel_time(distance, speed = 10):
    return distance/speed

In [35]:
class Ambulance(object):
    
    def __init__(self, base):
        
        #fix the ambulance's home base
        self.base = base
        
        #initialize the ambulance idle at home base
        
        #status code: 0-idle at base, 1-going to scene of call, 2-serving at scene of call, 3-going to hospital
        #4-transferring patient at hospital, 5-returning to base
        self.status = 0
        self.origin = base
        self.destination = base # if destination = origin, ambulance is stationary
        self.time = 0
        self.endtime = np.inf
        self.call = -1 # -1 if not assigned to any call
        
    def update_status(self, status, origin, destination, time, endtime):
        self.status = status
        
        self.origin = origin
        self.destination = destination
        self.time = time
        self.endtime = endtime
        
    def redployment(self):
        
        #current heuristic: return to home base
        distance = get_distance(self.origin, self.base)
        travel_time = get_ambulance_travel_time(distance)
        self.update_status(5, self.origin, self.base, copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        self.call = -1
        
    def return_to_base(self):
        self.update_status(0, self.base, self.base, copy.deepcopy(self.endtime), np.inf)
        
        
    def assign_to_call(self, call, time, index):
        
        distance = get_distance(self.origin, call.location)
        travel_time = get_ambulance_travel_time(distance)
        self.update_status(1, self.origin, call.location, time, time + travel_time)
        self.call = call.call_index # updated assigned call index
        
        call.status = index
        
        
    def reach_call_location(self, call, callList):
    
        atlocation_servicetime = gen_atlocation_service_time()
        self.update_status(2, call.location, call.location, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + atlocation_servicetime)

        callList.pop(call.call_index)
    
    def transport_to_hospital(self, city):
        
        #transport to nearest hospital
        
        hospital_list = city.hospital_location
        
        min_distance = city.width + city.height
        nearest_hospital = hospital_list[0]
        for hospital in hospital_list:
            distance = get_distance(self.origin, hospital)
            if distance < min_distance:
                min_distance = distance
                nearest_hospital = hospital
        
        travel_time = get_ambulance_travel_time(min_distance)
        self.update_status(3, self.origin, hospital, 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + travel_time)
        
    def reach_hospital(self):
        hospital_servicetime = gen_athospital_service_time()
        self.update_status(4, copy.deepcopy(self.destination), copy.deepcopy(self.destination), 
                           copy.deepcopy(self.endtime), copy.deepcopy(self.endtime) + hospital_servicetime)




In [36]:
def arrival(call_index, ambulanceList, callList, time_arrival, M, timeline):
    
    call = Call(call_index, gen_call_arrival_loc(city), time_arrival, gen_call_arrival_time())
    
    if len(callList) >= M:
        i = timeline.shape[0]
        timeline.loc[i] = [call_index, '', 7, time_arrival]
        print("New call arrived. No more capacity. Reject call.")
    
    else:
    
        print("New call arrived. Add to call list.")
#         callList.append(call)
        callList[call_index] = call

        min_distance = 15
        assign = -1
        index = 0

        nearest_distance = 15
        for ambulance in ambulanceList:

            if ambulance.status == 0:
                distance = get_distance(ambulance.origin, call.location)
                if distance < nearest_distance:
                    nearest_distance = distance
                    assign = index

            index = index + 1

        if assign > -1:
            # when the call arrives, there are ambulances idle at base, so assign the call to the nearest ambulance
            print("Idle ambulance at base available. Assign. Now travelling to call location.")
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, assign, 1, time_arrival]
            ambulanceList[assign].assign_to_call(call, time_arrival, assign)

        else:
            print("No available ambulance at the moment.")
            
    time_arrival = call.next_arrival_time
    
    return ambulanceList, callList, time_arrival, timeline

In [37]:
def get_next_event(time_arrival, ambulanceList, callList, city, M, timeline):
    
    ambulanceEndTime_min = np.inf
    index_min = -1
    index = 0
    for ambulance in ambulanceList:
        if ambulance.endtime < ambulanceEndTime_min:
            ambulanceEndTime_min = copy.deepcopy(ambulance.endtime)
            index_min = index
        
        index = index + 1
    
    next_event_time = min(time_arrival, ambulanceEndTime_min)
    print(time_arrival, ambulanceEndTime_min, next_event_time)
    if next_event_time == time_arrival:
        print("New call arrived.")
        i = timeline.shape[0]
        all_call = set(timeline['Call'])
        call_index = len(all_call) if '' not in all_call else len(all_call)-1
        timeline.loc[i] = [call_index, '', 0, time_arrival]
        ambulanceList, callList, time_arrival, timeline = arrival(call_index, ambulanceList, callList, time_arrival, M, timeline)
        
    else:
        if ambulanceList[index_min].status == 1:
            print("Now reach call location. Start at-location treatment. Remove call from call list.")
            call_index = ambulanceList[index_min].call
            call = callList[call_index]
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 2, next_event_time]
            ambulanceList[index_min].reach_call_location(call, callList)
            
        elif ambulanceList[index_min].status == 2:
            print("Now finish at-location treatment. Start going to hospital.")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 3, next_event_time]
            ambulanceList[index_min].transport_to_hospital(city)
            
        elif ambulanceList[index_min].status == 3:
            print("Now reach hospital. Start transferring patient to hospital.")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 4, next_event_time]
            ambulanceList[index_min].reach_hospital()
            
        elif ambulanceList[index_min].status == 4:
            
            print("Now finish transfering patient to hospital. Decide next step (assign to call or return to base).")
            call_index = ambulanceList[index_min].call
            i = timeline.shape[0]
            timeline.loc[i] = [call_index, index_min, 5, next_event_time]
            
            if len(callList) == 0:
                print("Return to base.")
                ambulanceList[index_min].redployment()
            else:
                print("Call waiting. Assign to the first unassigned call in queue.")
                assign = -1
                call_index = min(callList.keys())
                index = 0
                while assign == -1 and index < M:
                    
                    if call_index not in callList:
                        call_index += 1
                        continue
                    elif callList[call_index].status == -1:
                        assign = call_index
                        
                    call_index += 1
                    index = index + 1
                
                if index == M:
                    # calls in callList have all been assigned with an ambulance
                    ambulanceList[index_min].redployment()
                else:
                    i = timeline.shape[0]
                    timeline.loc[i] = [assign, index_min, 1, next_event_time]
                    print([assign, index_min, 1, next_event_time])
                    ambulanceList[index_min].assign_to_call(callList[assign], next_event_time, index_min)
                    
        elif ambulanceList[index_min].status == 5:
            i = timeline.shape[0]
            timeline.loc[i] = ['', index_min, 6, next_event_time]
            print("Now reployed ambulance reach base. Start idling.")
            ambulanceList[index_min].return_to_base()
    
    return time_arrival, ambulanceList, callList, timeline

In [38]:
def print_ambulance(ambulanceList):
    statusList = []
    originList = []
    destinationList = []
    timeList = []
    endtimeList = []
    
    for ambulance in ambulanceList:
        statusList.append(ambulance.status)
        originList.append(ambulance.origin)
        destinationList.append(ambulance.destination)
        timeList.append(ambulance.time)
        endtimeList.append(ambulance.endtime)
    
    return statusList, originList, destinationList, timeList, endtimeList
    

In [39]:
def print_call(callList):
    
    statusList = []
    locationList = []
    timeList = []
    
    for call_index in callList:
        call = callList[call_index]
        statusList.append(call.status)
        locationList.append(call.location)
        timeList.append(call.arrival_time)
    
    return statusList, locationList, timeList

In [40]:
def get_jobcount(timeline):
    timediff = np.append(np.diff(timeline['Timestamp']), 0)
    timeline['timediff'] = timediff
    
    n = timeline.shape[0]
    numCalls = np.zeros(n)
    
    count = 0
    for i in range(n):
        event = timeline.iloc[i]['Event']
        if event == 0:
            count += 1
        elif event == 5 or event == 7: 
            count -= 1
            
        if count <0:
            print("hi")
            
        numCalls[i] = count
        
    numCalls[-1] = numCalls[n-2]
    timeline['numCalls'] = numCalls
    return timeline

In [41]:
def get_jobs(timeline):
    total = max(timeline['Call']) # 
    
    arrival = np.zeros(total+1)*np.nan
    assign = np.zeros(total+1)*np.nan
    reach = np.zeros(total+1)*np.nan
    onsite = np.zeros(total+1)*np.nan
    transfer = np.zeros(total+1)*np.nan
    finish = np.zeros(total+1)*np.nan
    
    for i in range(total + 1):
        c = timeline[timeline['Call'] == i]
        n = c.shape[0]
#         print(n)
        for index, row in c.iterrows():
            t = row['Timestamp']
            event = row['Event']
            if event == 0:
                arrival[i] = t
            elif event == 1:
                assign[i] = t if n > 1 else np.nan
            elif event == 2:
                reach[i] = t if n > 2 else np.nan
            elif event == 3:
                onsite[i] = t if n > 3 else np.nan
            elif event == 4:
                transfer[i] = t if n > 4 else np.nan
            elif event == 5:
                finish[i] = t if n > 5 else np.nan
            elif event == 7:
                finish[i] = t
#         print(n, arrival[i], assign[i], reach[i], onsite[i], transfer[i], finish[i])
        
    columns = ['arrival_time', 'assigned_time', 'reach_patient', 'finish_onsite', 'reach_hospital', 'finish']
    data = list(zip(arrival, assign, reach, onsite, transfer, finish))
    df = pd.DataFrame(data, columns=columns)
    
    df['waiting_time'] = df['assigned_time'] - df['arrival_time']
    df['total_time'] = df['finish'] - df['arrival_time']
    return df

In [42]:
def performance_metric(c, timeline, df, target):
    t = timeline.iloc[-1]['Timestamp']
    P = timeline.groupby('numCalls')['timediff'].sum() / t
    
    expectn = sum(P * P.index)
    expectw = sum(P[c+1:] * (P.index[c:]-c))
    utilization = (expectn - expectw) / c
    
    print('Utilization:', utilization)
    print('Expected number of jobs in system:', expectn)
    print('Expected number of jobs in queue:', expectw)
    
    df_complete = df.dropna(axis=0)
    print('Expected time in queue:', np.mean(df_complete['waiting_time']))
    print('Expected time in system:', np.mean(df_complete['total_time']))
    
    assigned = df[df['assigned_time'] != np.nan]
    count = 0
    for index, row in assigned.iterrows():
        if np.isnan(row['total_time']) or row['total_time'] > target:
            count += 1
    
    print("Total unreachable calls:", count)   

# Set up the City

In [47]:
city = City(150, 150, 2, 4)
ambulanceList = [Ambulance(city.base_location[0]), Ambulance(city.base_location[1])]
callList = {}
time_arrival = 0
timeline = pd.DataFrame(columns = ['Call', 'Ambulance', 'Event', 'Timestamp'])

# Repetitively run the following two blocks to visualize the process

In [48]:
time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 10, timeline)
am_statusList, am_originList, am_destinationList, am_timeList, am_endtimeList = print_ambulance(ambulanceList)
cl_statusList, cl_locationList, cl_timeList = print_call(callList)

print("Next Arrival: ", time_arrival)

print("Ambulance List Print: ")
print("Status List: ", am_statusList)
print("Origin List: ", am_originList)
print("Destination List: ", am_destinationList)
print("Event start time List: ", am_timeList)
print("Event end time List: ", am_endtimeList)

print("Call List Print: ")
print("Status List: ", cl_statusList)
print("Location List: ", cl_locationList)
print("Arrival time List: ", cl_timeList)

0 inf 0
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
Next Arrival:  1.3853960989231748
Ambulance List Print: 
Status List:  [0, 0]
Origin List:  [(50.0, 50.0), (50.0, 100.0)]
Destination List:  [(50.0, 50.0), (50.0, 100.0)]
Event start time List:  [0, 0]
Event end time List:  [inf, inf]
Call List Print: 
Status List:  [-1]
Location List:  [(12.827034647628349, 107.63625238985098)]
Arrival time List:  [0]


In [49]:
T = 60*24*4

while timeline.iloc[-1]['Timestamp'] < T:
    time_arrival, ambulanceList, callList, timeline = get_next_event(time_arrival, ambulanceList, callList, city, 10, timeline)

1.3853960989231748 inf 1.3853960989231748
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
6.843464459289046 inf 6.843464459289046
New call arrived.
New call arrived. Add to call list.
Idle ambulance at base available. Assign. Now travelling to call location.
8.319002938474071 7.939096246253536 7.939096246253536
Now reach call location. Start at-location treatment. Remove call from call list.
8.319002938474071 28.707799775410166 8.319002938474071
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
9.319469333309662 28.707799775410166 9.319469333309662
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
10.284329190862525 28.707799775410166 10.284329190862525
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
11.327799829700636 28.707799775410166 11.327799829700636
New call arrived.
New call arrived. Add to call list.
No avai

New call arrived. No more capacity. Reject call.
207.76639208157235 207.89894264885987 207.76639208157235
New call arrived.
New call arrived. No more capacity. Reject call.
208.88660924106614 207.89894264885987 207.89894264885987
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[7, 1, 1, 207.89894264885987]
208.88660924106614 215.53682639258065 208.88660924106614
New call arrived.
New call arrived. No more capacity. Reject call.
214.13634187220228 215.53682639258065 214.13634187220228
New call arrived.
New call arrived. No more capacity. Reject call.
221.81190466123905 215.53682639258065 215.53682639258065
Now reach call location. Start at-location treatment. Remove call from call list.
221.81190466123905 226.62093226186605 221.81190466123905
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
221.90669179473917 226.62093226186605 221.90669

New call arrived. No more capacity. Reject call.
402.21571448972304 400.04395950451647 400.04395950451647
Now finish at-location treatment. Start going to hospital.
402.21571448972304 402.8388326746413 402.21571448972304
New call arrived.
New call arrived. No more capacity. Reject call.
403.34137545632814 402.8388326746413 402.8388326746413
Now reach hospital. Start transferring patient to hospital.
403.34137545632814 405.3309641421352 403.34137545632814
New call arrived.
New call arrived. No more capacity. Reject call.
403.97843547596904 405.3309641421352 403.97843547596904
New call arrived.
New call arrived. No more capacity. Reject call.
404.2254848850287 405.3309641421352 404.2254848850287
New call arrived.
New call arrived. No more capacity. Reject call.
407.97875187644496 405.3309641421352 405.3309641421352
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[41, 1, 1, 405.330

New call arrived. No more capacity. Reject call.
519.3800053443316 524.4053286365516 519.3800053443316
New call arrived.
New call arrived. No more capacity. Reject call.
523.0485024043683 524.4053286365516 523.0485024043683
New call arrived.
New call arrived. No more capacity. Reject call.
523.5331548597217 524.4053286365516 523.5331548597217
New call arrived.
New call arrived. No more capacity. Reject call.
531.8470375606017 524.4053286365516 524.4053286365516
Now reach call location. Start at-location treatment. Remove call from call list.
531.8470375606017 531.311889373924 531.311889373924
Now finish at-location treatment. Start going to hospital.
531.8470375606017 539.6985612005617 531.8470375606017
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
533.3522514173853 539.6985612005617 533.3522514173853
New call arrived.
New call arrived. No more capacity. Reject call.
538.843223410651 539.6985612005617 538.843223410651
New call arrived.
New 

New call arrived. Add to call list.
No available ambulance at the moment.
671.7022022869477 670.7660360224289 670.7660360224289
Now finish at-location treatment. Start going to hospital.
671.7022022869477 674.5752089894175 671.7022022869477
New call arrived.
New call arrived. No more capacity. Reject call.
672.1083483790158 674.5752089894175 672.1083483790158
New call arrived.
New call arrived. No more capacity. Reject call.
674.2342686167209 674.5752089894175 674.2342686167209
New call arrived.
New call arrived. No more capacity. Reject call.
682.8598301391459 674.5752089894175 674.5752089894175
Now reach hospital. Start transferring patient to hospital.
682.8598301391459 677.0113517022281 677.0113517022281
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[173, 1, 1, 677.0113517022281]
682.8598301391459 683.5887094861512 682.8598301391459
New call arrived.
New call arrived. No m

...

New call arrived. No more capacity. Reject call.
5156.9312795263095 5156.026283823647 5156.026283823647
Now reach hospital. Start transferring patient to hospital.
5156.9312795263095 5158.560584610911 5156.9312795263095
New call arrived.
New call arrived. No more capacity. Reject call.
5162.834146985261 5158.560584610911 5158.560584610911
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[1694, 1, 1, 5158.560584610911]
5162.834146985261 5163.222480262842 5162.834146985261
New call arrived.
New call arrived. No more capacity. Reject call.
5169.2034795370155 5163.222480262842 5163.222480262842
Now reach call location. Start at-location treatment. Remove call from call list.
5169.2034795370155 5163.342557914669 5163.342557914669
Now finish at-location treatment. Start going to hospital.
5169.2034795370155 5168.762818365173 5168.762818365173
Now reach call location. Start at-location 

5322.321896241178 5324.2358204788325 5322.321896241178
New call arrived.
New call arrived. No more capacity. Reject call.
5323.087875935449 5324.2358204788325 5323.087875935449
New call arrived.
New call arrived. No more capacity. Reject call.
5324.278469214835 5324.2358204788325 5324.2358204788325
Now reach hospital. Start transferring patient to hospital.
5324.278469214835 5326.677933528879 5324.278469214835
New call arrived.
New call arrived. No more capacity. Reject call.
5326.2446246162735 5326.677933528879 5326.2446246162735
New call arrived.
New call arrived. No more capacity. Reject call.
5330.103254039267 5326.677933528879 5326.677933528879
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[1742, 1, 1, 5326.677933528879]
5330.103254039267 5332.155190469557 5330.103254039267
New call arrived.
New call arrived. No more capacity. Reject call.
5330.395779983863 5332.155190469

New call arrived. No more capacity. Reject call.
5448.565660475788 5465.622362863695 5448.565660475788
New call arrived.
New call arrived. No more capacity. Reject call.
5453.53696203156 5465.622362863695 5453.53696203156
New call arrived.
New call arrived. No more capacity. Reject call.
5456.140369883205 5465.622362863695 5456.140369883205
New call arrived.
New call arrived. No more capacity. Reject call.
5462.902029276577 5465.622362863695 5462.902029276577
New call arrived.
New call arrived. No more capacity. Reject call.
5463.620394667283 5465.622362863695 5463.620394667283
New call arrived.
New call arrived. No more capacity. Reject call.
5465.819045239891 5465.622362863695 5465.622362863695
Now finish at-location treatment. Start going to hospital.
5465.819045239891 5472.698631050605 5465.819045239891
New call arrived.
New call arrived. No more capacity. Reject call.
5465.931182717265 5472.698631050605 5465.931182717265
New call arrived.
New call arrived. No more capacity. Reject

New call arrived. Add to call list.
No available ambulance at the moment.
5583.06807073659 5581.778965436838 5581.778965436838
Now reach hospital. Start transferring patient to hospital.
5583.06807073659 5584.278852520109 5583.06807073659
New call arrived.
New call arrived. No more capacity. Reject call.
5586.352210233887 5584.278852520109 5584.278852520109
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned call in queue.
[1807, 0, 1, 5584.278852520109]
5586.352210233887 5588.072457208506 5586.352210233887
New call arrived.
New call arrived. No more capacity. Reject call.
5588.036695844356 5588.072457208506 5588.036695844356
New call arrived.
New call arrived. No more capacity. Reject call.
5592.583846144098 5588.072457208506 5588.072457208506
Now finish at-location treatment. Start going to hospital.
5592.583846144098 5589.6514836782735 5589.6514836782735
Now reach hospital. Start transferring p

New call arrived. No more capacity. Reject call.
5708.448088857523 5711.285359378356 5708.448088857523
New call arrived.
New call arrived. No more capacity. Reject call.
5713.968369702336 5711.285359378356 5711.285359378356
Now reach call location. Start at-location treatment. Remove call from call list.
5713.968369702336 5714.900435759811 5713.968369702336
New call arrived.
New call arrived. Add to call list.
No available ambulance at the moment.
5719.199206887302 5714.900435759811 5714.900435759811
Now finish at-location treatment. Start going to hospital.
5719.199206887302 5719.3671814592735 5719.199206887302
New call arrived.
New call arrived. No more capacity. Reject call.
5721.977554420437 5719.3671814592735 5719.3671814592735
Now reach hospital. Start transferring patient to hospital.
5721.977554420437 5721.816855015287 5721.816855015287
Now finish transfering patient to hospital. Decide next step (assign to call or return to base).
Call waiting. Assign to the first unassigned c

In [50]:
timeline

,Call,Ambulance,Event,Timestamp
0,0,,0,0
1,1,,0,1.3854
2,2,,0,6.84346
3,2,1,1,6.84346
4,2,1,2,7.9391
...,...,...,...,...
5328,1889,1,1,5755
5329,1938,,0,5758.59
5330,1938,,7,5758.59
5331,1889,1,2,5759.38


In [51]:
timeline = get_jobcount(timeline)

In [52]:
timeline

,Call,Ambulance,Event,Timestamp,timediff,numCalls
0,0,,0,0,1.3854,1.0
1,1,,0,1.3854,5.45807,2.0
2,2,,0,6.84346,0,3.0
3,2,1,1,6.84346,1.09563,3.0
4,2,1,2,7.9391,0.379907,3.0
...,...,...,...,...,...,...
5328,1889,1,1,5755,3.59078,11.0
5329,1938,,0,5758.59,0,12.0
5330,1938,,7,5758.59,0.786192,11.0
5331,1889,1,2,5759.38,0.936578,11.0


In [53]:
df = get_jobs(timeline)

In [54]:
performance_metric(2, timeline, df, 5)

Utilization: 0.999879746440774
Expected number of jobs in system: 11.069725506467794
Expected number of jobs in queue: 9.069966013586246
Expected time in queue: 143.50204822099468
Expected time in system: 171.68514056266744
Total unreachable calls: 376


In [38]:
P = timeline.groupby('numCalls')['timediff'].sum() / timeline.iloc[-1]['Timestamp']

In [42]:
P

numCalls
1.0    0.000514
2.0    0.000766
3.0    0.011111
4.0    0.163294
5.0    0.824315
6.0    0.000000
Name: timediff, dtype: float64

In [51]:
P[3:]

numCalls
3.0    0.011111
4.0    0.163294
5.0    0.824315
6.0    0.000000
Name: timediff, dtype: float64

In [53]:
P.index[3:]

Float64Index([4.0, 5.0, 6.0], dtype='float64', name='numCalls')